### 0. Imports

In [11]:
%load_ext autoreload
%autoreload 2

# Data transformation
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualizations
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Progress loops
# -----------------------------------------------------------------------
from tqdm import tqdm

# Handle warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("once")

# modify system variables
# -----------------------------------------------------------------------
import sys
sys.path.append("..") # append parent folder to path

# import support functions
# -----------------------------------------------------------------------
import src.soporte_eda as se
import src.soporte_preprocesamiento as sp
import src.soporte_clustering as sc


# statistics functions
# -----------------------------------------------------------------------
from scipy.stats import pearsonr, spearmanr, pointbiserialr



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\Miguel\.virtualenvs\9._Clustering_-_Regresión-T2F1V_og\Lib\site-packages\yellowbrick\style\colors.py:35: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  mpl_ge_150 = LooseVersion(mpl.__version__) >= "1.5.0"
c:\Users\Miguel\.virtualenvs\9._Clustering_-_Regresión-T2F1V_og\Lib\site-packages\setuptools\_distutils\version.py:337: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
c:\Users\Miguel\.virtualenvs\9._Clustering_-_Regresión-T2F1V_og\Lib\site-packages\yellowbrick\style\rcmod.py:31: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  mpl_ge_150 = LooseVersion(mpl.__version__) >= "1.5.0"


# 1. Intro to this notebook and data

## 1.1 Introduction

The goal of this busines case is to 1. identify similar groups of clients according to their buying behaviour 2. identify groups of products according to their profitability. For that, analysis, clustering algorithms and regression models will be used.

The purpose of this notebook is to explore Global Ecommerce's product and customers data in order to clean any possible errors that could impair the extraction of insights through analysis and the quality of the subsequent model development for their brand.

## 1.2 Data import

In [10]:
global_superstore = pd.read_csv("../data/Global_Superstore.csv", encoding="latin1")

# transform column names into lowercase and replace spaces by underscores
global_superstore.columns = [col.lower().replace(" ", "_") for col in global_superstore.columns]

global_superstore.head(3)

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,...,product_id,category,sub-category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
0,32298,CA-2012-124891,31-07-2012,31-07-2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,05-02-2013,07-02-2013,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,17-10-2013,18-10-2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium


# 2. Preliminary data analysis and cleaning

For a quick initial exploration:

In [15]:
se.exploracion_dataframe(global_superstore)

El número de datos es 51290 y el de columnas es 24

 ..................... 

5 filas aleatorias del dataframe son:


,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,...,product_id,category,sub-category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
33800,19820,ES-2011-4436456,26-01-2011,26-01-2011,Same Day,HG-15025,Hunter Glantz,Consumer,Reims,Champagne-Ardenne,...,OFF-BI-10001384,Office Supplies,Binders,"Cardinal Binder, Economy",41.4900,6,0.50,-10.8900,3.98,Medium
6935,20592,ID-2013-22025,09-10-2013,13-10-2013,Standard Class,TS-21655,Trudy Schmidt,Consumer,Bandung,Jawa Barat,...,TEC-CO-10000821,Technology,Copiers,"Canon Ink, High-Speed",675.8775,5,0.07,-43.6725,49.27,High
1653,19562,ES-2011-3142386,06-12-2011,06-12-2011,Same Day,LT-17110,Liz Thompson,Consumer,Hamburg,Hamburg,...,TEC-PH-10001066,Technology,Phones,"Apple Smart Phone, Cordless",636.3000,1,0.00,139.9800,151.11,High
25870,2861,MX-2014-130295,28-04-2014,30-04-2014,Second Class,SB-20185,Sarah Brown,Consumer,Granada,Granada,...,TEC-PH-10004242,Technology,Phones,"Apple Speaker Phone, VoIP",82.2600,1,0.00,18.0800,7.66,High
7875,3635,MX-2013-137449,04-11-2013,11-11-2013,Standard Class,RP-19855,Roy Phan,Corporate,Guantánamo,Guantánamo,...,FUR-FU-10001142,Furniture,Furnishings,"Deflect-O Clock, Black",234.7800,7,0.00,35.1400,43.05,Low



 ..................... 

Los tipos de las columnas y sus valores únicos son:


,tipo_dato,conteo
row_id,int64,51290
order_id,object,25035
order_date,object,1430
ship_date,object,1464
ship_mode,object,4
customer_id,object,1590
customer_name,object,795
segment,object,3
city,object,3636
state,object,1094



 ..................... 

Los duplicados que tenemos en el conjunto de datos son: 0

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


...

,count,pct
product_name,,
Staples,227,0.4
"Cardinal Index Tab, Clear",92,0.2
"Eldon File Cart, Single Width",90,0.2
"Rogers File Cart, Single Width",84,0.2
"Ibico Index Tab, Clear",83,0.2


La columna QUANTITY tiene 14 valores únicos, de los cuales los primeros son:


,count,pct
quantity,,
2,12748,24.9
3,9682,18.9
1,8963,17.5
4,6385,12.4
5,4882,9.5


La columna ORDER_PRIORITY tiene 4 valores únicos, de los cuales los primeros son:


,count,pct
order_priority,,
Medium,29433,57.4
High,15501,30.2
Critical,3932,7.7
Low,2424,4.7


**Aggregation level**

First and foremost, it necessary to notice that the data at hand consists of order records data. To use this data to analyse by clients or to analyse by products, agggregation on the appropiate level will be necessary.